<a href="https://colab.research.google.com/github/TrouHo/-USTH-Blood-Cells-Detection/blob/main/Faster_RCNN_Model___Blood_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Download dataset

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
!cp /path/to/dataset/on/your/drive .

Mounted at /content/drive
cp: cannot stat '/path/to/dataset/on/your/drive': No such file or directory


In [ ]:
# https://drive.google.com/drive/folders/1M3kDXzsLLCJrTqg8ThP8bncawEbkE7Ts?usp=sharing
!gdown '1M3kDXzsLLCJrTqg8ThP8bncawEbkE7Ts'

/usr/local/lib/python3.10/dist-packages/gdown/download.py:33: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs4.BeautifulSoup(line, features="html.parser")
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1M3kDXzsLLCJrTqg8ThP8bncawEbkE7Ts

but Gdown can't. Please check connections and permissions.


2. Import libary and framework required


In [ ]:
!pip install pandas matplotlib tensorflow==2.8.0 keras==2.8.0 numpy opencv-python scikit-learn h5py


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 43.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

3. Implement Faster RCNN

In [ ]:
!git clone https://github.com/kbardool/Keras-frcnn.git

Cloning into 'Keras-frcnn'...
remote: Enumerating objects: 635, done.
remote: Total 635 (delta 0), reused 0 (delta 0), pack-reused 635 (from 1)
Receiving objects: 100% (635/635), 187.79 KiB | 4.08 MiB/s, done.
Resolving deltas: 100% (433/433), done.


4.1 Convert dataset for training

In [ ]:
%pylab inline

import os, sys, random
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile

annotations = glob('BCCD_Dataset-master/BCCD/Annotations/*.xml')

df = []
cnt = 0

for file in annotations:
  prev_filename = file.split('/')[-1].split('.')[0] + '.jpg'
  filename = str(cnt) + '.jpg'
  row = []
  parsedXML = ET.parse(file)

  for node in parsedXML.getroot().iter('object'):
      blood_cells = node.find('name').text
      xmin = int(node.find('bndbox/xmin').text)
      xmax = int(node.find('bndbox/xmax').text)
      ymin = int(node.find('bndbox/ymin').text)
      ymax = int(node.find('bndbox/ymax').text)

  row = [prev_filename, blood_cells, xmin, xmax, ymin, ymax]
  df.append(row)
  cnt += 1

data = pd.DataFrame(df, columns=['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax'])

data[['filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv('blood_cell_detection.csv', index=False)


Populating the interactive namespace from numpy and matplotlib


4.2. Conver dataset again

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches

train = pd.read_csv('blood_cell_detection.csv')
train.head()

data = pd.DataFrame()
data['format'] = train['filename']

for i in range(data.shape[0]):
  data['format'][i] = 'train_images/' + data['format'][i]

for i in range(data.shape[0]):
  data['format'][i] = data['format'][i] + ',' + str(train['xmin'][i]) + ',' + str(train['ymin'][i]) + ',' + str(train['xmax'][i]) + ',' + str(train['ymax'][i]) + ',' + train['cell_type'][i]

data.to_csv('annotate.txt', header=None, index=None, sep=' ')


In [ ]:
5. Training

5.1 Import file pickle

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
!cp /path/to/dataset/on/your/drive .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/path/to/dataset/on/your/drive': No such file or directory


In [ ]:
# https://drive.google.com/file/d/16eB-kd7o--AMZiFMg-rpYOKDvZv3G7D0/view?usp=drive_link

!gdown '16eB-kd7o--AMZiFMg-rpYOKDvZv3G7D0'

Downloading...
From: https://drive.google.com/uc?id=16eB-kd7o--AMZiFMg-rpYOKDvZv3G7D0
To: /content/config.pickle
100% 840/840 [00:00<00:00, 2.71MB/s]


5.2. Train model

In [ ]:
%cd Keras-frcnn

/content/Keras-frcnn


In [ ]:
!python train_frcnn.py -o simple -p annotate.txt


  File "/content/Keras-frcnn/train_frcnn.py", line 112
    print(f'Num train samples {len(train_imgs}')
                                               ^
SyntaxError: f-string: closing parenthesis '}' does not match opening parenthesis '('
